In [1]:
import xarray as xr
import requests
import boto3
import s3fs
import rasterio as rio
import rioxarray
from rasterio.session import AWSSession
import os
from netrc import netrc
from subprocess import Popen
from platform import system
from getpass import getpass

In [2]:
# Generates authentication token
# Asks for your Earthdata username and password for first time, if netrc does not exists.

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL endpoint for authentication
prompts = ['Enter NASA Earthdata Login Username: ',
           'Enter NASA Earthdata Login Password: ']

# Determine the OS (Windows machines usually use an '_netrc' file)
netrc_name = "_netrc" if system()=="Windows" else ".netrc"

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser(f"~/{netrc_name}")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}{2} | echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)
    # Set restrictive permissions
    Popen('chmod 0600 {0}{1}'.format(homeDir + os.sep, netrc_name), shell=True)

    # Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)



In [3]:
# Generates the temporary
s3_cred_endpoint='https://archive.podaac.earthdata.nasa.gov/s3credentials'
 
def get_temp_creds():
    temp_creds_url=s3_cred_endpoint
    return requests.get(temp_creds_url).json()
 
temp_creds_req=get_temp_creds()
 
fs_s3 = s3fs.S3FileSystem(anon=False,
                          key=temp_creds_req['accessKeyId'],
                          secret=temp_creds_req['secretAccessKey'],
                          token=temp_creds_req['sessionToken'],
                          client_kwargs={'region_name':'us-west-2'})
 
with fs_s3.open('s3://podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_PROVISIONAL_V0/OPERA_L3_DSWx-HLS_T42RUQ_20220930T055641Z_20230112T024042Z_S2A_30_v0.0_B01_WTR.tif', mode='rb') as s3obj:
    
    # open with rasterio directly
    with rio.open(s3obj) as ds:
       print(ds.crs)
   
    # open with xarray
    B01_T42RUQ_S3=xr.open_dataset(s3obj, engine='rasterio')

EPSG:32642


In [4]:
B01_T42RUQ_S3

<xarray.Dataset>
Dimensions:      (band: 1, x: 3660, y: 3660)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 3e+05 3e+05 3.001e+05 ... 4.098e+05 4.098e+05
  * y            (y) float64 3e+06 3e+06 3e+06 ... 2.89e+06 2.89e+06 2.89e+06
    spatial_ref  int64 ...
Data variables:
    band_data    (band, y, x) float32 ...